In [32]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import tensorflow as tf

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import plotly.express as px
plt.style.use('ggplot')

# Load Data

In [ ]:
#!wget https://github.com/isaacmenchaca97/hospital_wait_time_prediction/releases/download/v1.0.0/ARTICLE.tar.gz
# !wget https://github.com/isaacmenchaca97/hospital_wait_time_prediction/releases/download/v1.0.0/EDIESCA.tar.gz

#!tar -xzvf ARTICLE.tar.gz
# !tar -xzvf EDIESCA.tar.gz

In [36]:
# Select target and features
df_analisys = pd.read_csv('../data/interim/data_analisys.csv')
df_analisys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215177 entries, 0 to 215176
Data columns (total 17 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Fecha                        215177 non-null  object 
 1   apat                         215177 non-null  object 
 2   amat                         215177 non-null  object 
 3   nombre                       215177 non-null  object 
 4   tiempo_espera_triage         215177 non-null  float64
 5   tiempo_en_triage             215177 non-null  float64
 6   tiempo_espera_despuestriage  215177 non-null  float64
 7   tiempo_en_consulta           215177 non-null  float64
 8   clasificacion                215177 non-null  object 
 9   edad                         215177 non-null  float64
 10  edad_unidad                  215177 non-null  float64
 11  Dx                           215177 non-null  object 
 12  tiempo_total                 215177 non-null  float64
 13 

# Analisys

##### Prepare data

In [37]:
df_analisys['date'] = pd.to_datetime(df_analisys['Fecha'])
df_analisys['entry_time'] = df_analisys['date'].dt.time
df_analisys['entry_time'] = pd.to_timedelta(df_analisys.entry_time.astype(str))


df_analisys['waiting_time'] = pd.to_timedelta(df_analisys['tiempo_total'], unit='m')

df_analisys['completion_time'] = df_analisys['waiting_time'] + df_analisys['entry_time']

df_analisys['waiting_ber_munets'] = df_analisys.waiting_time.dt.seconds / 60
df_analisys['waiting_ber_munets'] = df_analisys['waiting_ber_munets'].round(0)

df_analisys['weekday'] = df_analisys.date.dt.strftime('%A')                      # new column for extract 'weekday' from 'date'
df_analisys['hours'] = df_analisys.entry_time.dt.components.hours                # new column for extract the  'hours' from 'entry_time'

In [38]:
df_analisys.drop(columns=['nombre', 'genero', 'Dx'], inplace=True)

In [39]:
df_analisys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215177 entries, 0 to 215176
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype          
---  ------                       --------------   -----          
 0   Fecha                        215177 non-null  object         
 1   apat                         215177 non-null  object         
 2   amat                         215177 non-null  object         
 3   tiempo_espera_triage         215177 non-null  float64        
 4   tiempo_en_triage             215177 non-null  float64        
 5   tiempo_espera_despuestriage  215177 non-null  float64        
 6   tiempo_en_consulta           215177 non-null  float64        
 7   clasificacion                215177 non-null  object         
 8   edad                         215177 non-null  float64        
 9   edad_unidad                  215177 non-null  float64        
 10  tiempo_total                 215177 non-null  float64        
 11  clasificacion

##### Patient heatmap

In [53]:
def grouped_data2(column_name):
    if column_name == 'completion_time':
        # For completion_time, we want to count occurrences
        answer = pd.pivot_table(df_analisys, index='hours', 
                              columns=['weekday'], 
                              values=column_name,  # Specify which column to count
                              aggfunc='count')
    elif column_name == 'waiting_ber_munets':
        # For waiting time, we want to calculate mean
        answer = pd.pivot_table(df_analisys, index='hours',
                              columns=['weekday'], 
                              values='waiting_ber_munets', 
                              aggfunc='mean').round(1)
    else:
        return(column_name + " is not in the columns")
    
    # Fill any missing values with 0
    answer = answer.fillna(0)
    
    # Reorder columns to ensure consistent weekday order
    weekdays = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
    answer = answer.reindex(columns=weekdays)
    
    return answer

answer3 =  grouped_data2('completion_time')
answer4 =  grouped_data2('waiting_ber_munets')

In [51]:
fig3 = px.imshow(answer3,
                labels=dict(x="weekday", y="hours", color="number of paitant") ,
                aspect="auto", color_continuous_scale='tempo',
                template = template_style,
                text_auto=True, width=900, height=900)
fig3.update_xaxes(side="top")
fig3.show()

##### Hour heatmap

In [52]:
fig4 = px.imshow(answer4,
                labels=dict(x="weekday", y="hours",
                            color="the waiting time per min") ,
                            aspect="auto", color_continuous_scale='tempo',
                            template = template_style,
                            text_auto=True, width=900, height=900)
fig4.update_xaxes(side="top")
fig4.show()

##### Bar chart

In [46]:
def grouped_data(column_name):
    '''
    Groupby column and return DataFrame
    Input: Column Name
    '''
    df_tmp = df_analisys.groupby(column_name)[['waiting_ber_munets']].mean().round(0)
    df_tmp2 = df_analisys[column_name].value_counts()      #this for concat

    pivot_f = pd.concat([df_tmp, df_tmp2.rename('number_of_patient')],axis=1)
    pivot_f.reset_index(inplace=True)
    pivot_f = pivot_f.rename(columns={'index': column_name })
    return pivot_f

answer5 = grouped_data('hours')
link_size = [3,6]
fig = px.bar(x=answer5['hours'],
             y=answer5['waiting_ber_munets'],
             template= template_style,
             text_auto='.2s',
             labels={'x':'the hour',
                     'y':'the waiting time per m'}
             ).add_traces(
      px.line(answer5, x=answer5['hours'], text='number_of_patient',
             y=answer5['number_of_patient'],markers=True).update_traces(yaxis="y2",
             showlegend=True, line=dict(color = 'red', width=link_size[1]), name="number_of_patient").data)
fig.update_layout(yaxis2={"side":"right", "overlaying":"y"})
fig.show()

##### Weekly chart

In [47]:
answer2 =  grouped_data('weekday')
# Create Chart for the Daily
fig2 = px.bar(answer2,
             x='weekday',
             y='number_of_patient',
             color='waiting_ber_munets',
             labels={'waiting_ber_munets':'the waiting time per m'} ,
             color_continuous_scale=['green','yellow','red'],
             template = template_style,
             title = '<b>Daily visualization</b>')
# Display Plot
fig2.show()